In [1]:
import xml.etree.ElementTree as et
import os
import pandas as pd
import numpy as np

# Load Data

In [2]:
writers_path = "writers.pkl.compressed"
if os.path.exists(writers_path):
    writers = pd.read_pickle(writers_path, compression="gzip")
else:
    writers = pd.DataFrame(
        {
            "handedness": pd.Series(dtype=object),
            "sex": pd.Series(dtype=object),
            "education": pd.Series(dtype=object),
            "birth_year": pd.Series(dtype="int32"),
            "word_count": pd.Series(dtype="int32"),
        }
    )
    for root, dirs, files in os.walk("Writers/"):
        for filename in files:
            file_path = os.path.join(root, filename)
            xml = et.parse(file_path).getroot()

            id = filename[:-4]
            handedness = xml.find("handedness").text
            sex = xml.find("sex")
            if sex is not None:
                sex = sex.text
            education = xml.find("education")
            if education is not None:
                education = education.text
            birth_year = xml.find("birthYear")
            if birth_year is not None:
                birth_year = int(birth_year.text)

            writers.loc[id] = {
                "handedness": handedness,
                "sex": sex,
                "education": education,
                "birth_year": birth_year,
                "word_count": 0,
            }
    writers.to_pickle(writers_path, compression="gzip")
writers


,handedness,sex,education,birth_year,word_count
00136a23-d38a-4e79-847f-e6ef91ced567,Right,Woman,Master,1374,0
002c2b0b-afc0-4491-afbc-08bea8fb837f,Right,Woman,Doctoral,1367,0
00767657-9969-4781-b4f9-6430ee1a8077,Right,Man,Master,1376,0
007cfec5-01f1-40c8-8c99-ff5fd4076cd8,Right,Man,Bachelor,1380,0
00806490-2371-478c-a820-76bdb049075a,Right,Man,UpperSecondary,1380,0
...,...,...,...,...,...
fe35f6fe-4003-4f7b-8b8b-9eda43251ca3,Right,Woman,Bachelor,1379,0
fe5cace8-0e34-4079-8276-129a4aa02c9d,Right,Man,Bachelor,1370,0
fe8be55e-b3e5-437a-9e3a-1cf8f626af6a,Left,Man,Bachelor,1376,0
ff694413-bcc8-4657-afc3-3d07e3bc800e,Right,Man,Bachelor,1378,0


In [3]:
ground_truths_path = "ground_truths.pkl.compressed"
if os.path.exists(ground_truths_path):
    ground_truths = pd.read_pickle(ground_truths_path, compression="gzip")
else:
    ground_truths = pd.DataFrame(
        {
            "word_count": pd.Series(dtype="int32"),
        }
    )
    for root, dirs, files in os.walk("GroundTruths/"):
        for filename in files:
            file_path = os.path.join(root, filename)
            xml = et.parse(file_path).getroot()

            for text in xml.findall("text"):
                id = int(text.attrib["id"])
                if xml.attrib["type"] == "Text":
                    word_count = len(text.find("content").text.split(" "))
                else:
                    word_count = len(text.findall(".//line"))

                ground_truths.loc[id] = {
                    "word_count": word_count,
                }
    ground_truths.to_pickle(ground_truths_path, compression="gzip")
ground_truths


,word_count
250046,14
250047,14
250048,14
250049,14
250050,14
...,...
230041,7
230042,7
230043,7
230044,9


In [10]:
xml_namespaces = {"inkml": "http://www.w3.org/2003/InkML"}
writepads_path = "writepads.pkl.compressed"
if os.path.exists(writepads_path):
    writepads = pd.read_pickle(writepads_path, compression="gzip")
else:
    writepads = pd.DataFrame(
        {
            "writer_id": pd.Series(dtype=object),
            "input": pd.Series(dtype=object),
            "type": pd.Series(dtype=object),
            "hand": pd.Series(dtype=object),
            "truth_id": pd.Series(dtype="int32"),
        }
    )
    for root, dirs, files in os.walk("Writepads/"):
        if os.path.basename(root) == "Sign":
            continue
        for filename in files:
            file_path = os.path.join(root, filename)
            ink = et.parse(file_path).getroot()

            id = int(filename[:-6])
            writer_id = ink.find("inkml:annotationXML/writerId", xml_namespaces).text[9:]
            input = ink.find("inkml:annotationXML/input", xml_namespaces).text
            type = ink.find("inkml:annotationXML/type", xml_namespaces).text
            hand = ink.find("inkml:annotationXML/hand", xml_namespaces).text
            truth_id = int(ink.find("inkml:annotationXML/truthId", xml_namespaces).text)

            writers.loc[writer_id, "word_count"] += ground_truths.loc[truth_id, "word_count"]

            writepads.loc[id] = {
                "writer_id": writer_id,
                "input": input,
                "type": type,
                "hand": hand,
                "truth_id": truth_id,
            }
    writers.to_pickle(writers_path, compression="gzip")
    writepads.to_pickle(writepads_path, compression="gzip")
writepads


,writer_id,input,type,hand,truth_id
12088,e6b003ad-19a1-4422-81b6-c9973f542c12,Pen,NumberGroup,Right,250058
12093,92bacc45-ecd3-4cb5-9f16-9b8c176cc3a2,Mouse,NumberGroup,Right,250046
12101,c0482d8c-48d2-483b-9a45-6ef156a8033b,Mouse,NumberGroup,Right,250090
12102,c0482d8c-48d2-483b-9a45-6ef156a8033b,Mouse,NumberGroup,Right,250091
12340,a314eb9f-c743-432a-8695-c4901c7fe35a,Touch,NumberGroup,Right,250058
...,...,...,...,...,...
16513,b2caf855-9842-453f-b609-ee9e99bb41a0,Touch,WordGroup3,Right,210256
16514,b2caf855-9842-453f-b609-ee9e99bb41a0,Touch,WordGroup3,Right,210257
16515,b2caf855-9842-453f-b609-ee9e99bb41a0,Touch,WordGroup3,Right,210258
16516,b2caf855-9842-453f-b609-ee9e99bb41a0,Touch,WordGroup3,Right,210259


# Utils

In [11]:
def calculate_value_counts(data, labels):
    value_counts = data.value_counts(dropna=False).sort_index().reset_index(drop=True)
    value_counts *= 100 / np.sum(value_counts)
    value_counts = value_counts.round().astype(int)
    value_index = value_counts.index
    result = ''
    for i in range(len(labels)):
        value = 0
        if i in value_index:
            value = value_counts.loc[i]
        result += f'{labels[i]}: {value}%\n'
    return result[:-1]

def calculate_value_counts_direct(value_counts, labels):
    value_counts *= 100 / np.sum(value_counts)
    value_counts = value_counts.round().astype(int)
    value_index = value_counts.index
    result = ''
    for i in range(len(labels)):
        value = 0
        if i in value_index:
            value = value_counts.loc[i]
        result += f'{labels[i]}: {value}%\n'
    return result[:-1]

# Stats

### General Counts

In [12]:
print(f"Total Word Count: {writers['word_count'].sum()}")
valid_writers = writers[writers["word_count"] > 0]
print(f"Total Writer Count: {len(valid_writers)}")

Total Word Count: 113682
Total Writer Count: 372


### Type

In [15]:
labels = ['Text', 'WordGroup', 'WordGroup2', 'WordGroup3', 'NumberGroup']
counts = [0, 0, 0, 0, 0]
for w in writepads.values:
    input = w[2]
    for i in range(len(labels)):
        if input == labels[i]:
            counts[i] += ground_truths.loc[w[4], "word_count"]
print(calculate_value_counts_direct(pd.Series(counts), labels))

Text: 54%
WordGroup: 36%
WordGroup2: 2%
WordGroup3: 6%
NumberGroup: 2%


### Input

In [16]:
labels = ['Mouse', 'Pen', 'Touch', 'Touchpad', 'TouchPen']
counts = [0, 0, 0, 0, 0]
for w in writepads.values:
    input = w[1]
    for i in range(len(labels)):
        if input == labels[i]:
            counts[i] += ground_truths.loc[w[4], "word_count"]
print(calculate_value_counts_direct(pd.Series(counts), labels))

Mouse: 17%
Pen: 25%
Touch: 40%
Touchpad: 2%
TouchPen: 16%


### Hand

In [17]:
labels = ['Left', 'Right']
counts = [0, 0]
for w in writepads.values:
    input = w[3]
    for i in range(len(labels)):
        if input == labels[i]:
            counts[i] += ground_truths.loc[w[4], "word_count"]
print(calculate_value_counts_direct(pd.Series(counts), labels))

Left: 12%
Right: 88%


### Handedness

In [18]:
labels = sorted(['Both', 'Left', 'Right'])
print(calculate_value_counts(valid_writers["handedness"], labels))

Both: 1%
Left: 14%
Right: 85%


### Sex

In [19]:
labels = sorted(['Man', 'Woman']) + ['Unknown']
print(calculate_value_counts(valid_writers["sex"], labels))

Man: 63%
Woman: 35%
Unknown: 2%


### Education

In [20]:
labels = (
    sorted(
        [
            "Bachelor",
            "Doctoral",
            "LowerSecondary",
            "Master",
            "None",
            "Primary",
            "ShortCycleTertiary",
            "UpperSecondary",
        ]
    )
    + ["Unknown"]
)
print(calculate_value_counts(valid_writers["education"], labels))


Bachelor: 44%
Doctoral: 4%
LowerSecondary: 3%
Master: 27%
None: 1%
Primary: 1%
ShortCycleTertiary: 5%
UpperSecondary: 14%
Unknown: 2%


### Word - Writer

In [21]:
bins = [0, 150, 300, 600, 1200, 2400]
hist = np.histogram(valid_writers["word_count"], bins=bins)
user = np.append(hist[0], len(valid_writers[valid_writers["word_count"] >= 2400]))
hist = (user, bins)
result = 'word, writer\n'
labels = []
for i in range(len(user)):
    if bins[i] == 2400:
        labels.append('2400+')
    else:
        labels.append(f'{bins[i]}')
    result += f'{labels[i]}, '
    result += f'{user[i]:.0f}\n'
print('Normal')
print(result[:-1])

sum = 0
for i in reversed(range(len(user))):
    sum += user[i]
    user[i] = sum
result = 'word, writer\n'
labels = []
for i in range(len(user)):
    if bins[i] == 2400:
        labels.append('2400+')
    else:
        labels.append(f'{bins[i]}')
    result += f'{labels[i]}, '
    result += f'{user[i]:.0f}\n'
print()
print('Aggregated')
print(result[:-1])

Normal
word, writer
0, 210
150, 34
300, 69
600, 47
1200, 9
2400+, 3

Aggregated
word, writer
0, 372
150, 162
300, 128
600, 59
1200, 12
2400+, 3


### Age - Writer

In [22]:
year_users = 1401 - valid_writers['birth_year'][pd.notna(valid_writers['birth_year'])]
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
hist = np.histogram(year_users, bins=bins)
user = np.append(hist[0], len(year_users[year_users >= 100]))
hist = (user, bins)
result = 'age, writer\n'
labels = []
for i in range(len(hist[0])):
    if bins[i] == 100:
        labels.append('100+')
    else:
        labels.append(f'{bins[i]}')
    result += f'{labels[i]}, '
    result += f'{user[i]}\n'
print(result)

age, writer
0, 0
10, 49
20, 216
30, 64
40, 24
50, 2
60, 1
70, 0
80, 1
90, 0
100+, 6

